In [17]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)
pd.set_option("display.min_rows", 200)
pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = None



In [18]:
ddf = pd.read_csv('../data/indiana/indiana_guardianship_dockets.csv')
cdf = pd.read_csv('../data/indiana/indiana_clean_agc_filings.csv')

## Cleaned Dataset EDA

In [3]:
# how many neither approved or denied

missing_outcome_casenums = cdf[~cdf.guardianship_approved & ~cdf.guardianship_denied].case_number.unique().tolist()

In [8]:
print('Percent missing disposition:', round(len(missing_outcome_casenums)/cdf.shape[0]*100, 1), '%')

Percent missing disposition: 32.3 %


## Docket EDA

In [5]:
ddf.columns

Index(['c2dp_case_id', 'fips', 'case_key', 'state', 'county', 'case_number',
       'raw_case_type', 'clean_case_type', 'date_filed', 'date_closed',
       'date_disposed', 'disposition', 'raw_case_status', 'clean_case_status',
       'is_closed', 'docket_eventkey', 'docket_casekey',
       'docket_baseeventtype', 'docket_baseeventtypekey', 'docket_eventtype',
       'docket_event_date', 'docket_event_description'],
      dtype='object')

In [6]:
NUM_OF_EXAMPLES = 10
def get_docket_entry_data(df):
    data = {}
    data['count'] = df.case_number.nunique()
    data['case_examples'] = "; ".join(df.sample(NUM_OF_EXAMPLES, replace=True).case_number.unique().tolist())
    return pd.Series(data)

frequency_table = ddf.groupby('docket_event_description').apply(get_docket_entry_data).sort_values('count', ascending=False).reset_index()
frequency_table.to_csv(f'../data/indiana/docket_keyword_search_results/all.csv')

### Comment EDA

In [ ]:
dates or numbers/symbols generally
names
----

In [34]:
ddf['clean_comment'] = ddf.docket_event_comment.str.lower()
print('Variations before:', ddf.clean_comment.nunique())
erase_list = ['----.*', '[\d\/\,\.]', '\: *(am|pm)', '\n']
for e in erase_list:
    ddf['clean_comment'] = ddf['clean_comment'].str.replace(e, ' ', regex=True)
print('Variations after cleaning:', ddf.clean_comment.nunique())

Variations before: 146739
Variations after cleaning: 73427


In [35]:
ddf[~ddf.docket_event_comment.isnull()].sample(100)

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description,docket_event_comment,clean_comment
247261,02D03-1902-GU-000018,18003,indiana___02D03-1902-GU-000018,indiana,allen,02d03-1902-gu-000018,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2019-02-06,2019-04-03,NaN,NaN,decided,disposed/closed,True,496901920,34700874,C,OTHER,ENITPB,02/22/2019,Automated ENotice Issued to Parties,Correspondence to/from Court Filed ---- 2/21/2019 : Paul Richard Rauch;Stacy Lynn McGuyre,correspondence to from court filed
233965,49D08-1901-GU-001314,18097,indiana___49D08-1901-GU-001314,indiana,marion,49d08-1901-gu-001314,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2019-01-10,2021-03-29,NaN,NaN,decided,disposed/closed,True,571820627,34622476,C,OTHER,DOC,07/07/2020,Document Filed,Application for Appointment of Guardian - Kimberly Shelby,application for appointment of guardian - kimberly shelby
57261,53C01-1808-GU-000113,18105,indiana___53C01-1808-GU-000113,indiana,monroe,53c01-1808-gu-000113,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-08-17,2021-07-07,NaN,NaN,decided,disposed/closed,True,468403911,33560130,C,C,ADMOAF,08/17/2018,Oath Filed,Acceptance and Oath of Temporary Guardian,acceptance and oath of temporary guardian
224125,77C01-1903-GU-000014,18153,indiana___77C01-1903-GU-000014,indiana,sullivan,77c01-1903-gu-000014,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2019-03-12,NaN,NaN,NaN,pending,open/pending,False,660679609,35649866,C,OTHER,ENITPB,12/16/2021,Automated ENotice Issued to Parties,Notice Issued to Parties ---- 12/15/2021 : Ashley Mae Dyer;Lucas John Rowe,notice issued to parties
74496,53C01-1710-GU-000141,18105,indiana___53C01-1710-GU-000141,indiana,monroe,53c01-1710-gu-000141,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-10-02,2021-03-17,NaN,NaN,decided,disposed/closed,True,398389162,28651338,C,OTHER,ENITPB,12/06/2017,Automated ENotice Issued to Parties,Hearing Journal Entry ---- 12/5/2017 : Evelyn Harrell;Elizabeth Ruh;Robert Thomas Miller\nOrder Issued ---- 12/5/2017 : Evelyn Harrell;Elizabeth Ruh;Robert Thomas Miller\nOrder Issued ---- 12/5/2017 : Evelyn Harrell;Elizabeth Ruh;Robert Thomas Miller\nOrder Issued ---- 12/5/2017 : Evelyn Harrell;Elizabeth Ruh;Robert Thomas Miller,hearing journal entry order issued order issued order issued
47100,59C01-1710-GU-000055,18117,indiana___59C01-1710-GU-000055,indiana,orange,59c01-1710-gu-000055,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-10-04,2018-12-05,NaN,NaN,decided,disposed/closed,True,399935084,28660250,C,OTHER,ENITPB,12/28/2017,Automated ENotice Issued to Parties,Hearing Scheduling Activity ---- 12/27/2017 : Marilyn Alice Tucker Fullen,hearing scheduling activity
18707,88C01-1605-GU-000023,18175,indiana___88C01-1605-GU-000023,indiana,washington,88c01-1605-gu-000023,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2016-05-13,2016-07-13,NaN,NaN,decided,disposed/closed,True,308368125,23219583,C,C,APP,05/13/2016,Appearance Filed,GR - Filed electronically,gr - filed electronically
138685,05C01-1805-GU-000007,18009,indiana___05C01-1805-GU-000007,indiana,blackford,05c01-1805-gu-000007,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2018-05-29,2018-11-29,NaN,NaN,decided,disposed/closed,True,461391817,32397358,C,C,SRS,08/01/2018,Service Returned Served,Notice of Filing of Petition for Appointment of Co-Guardians and Hearing Thereon served by Hancock Cty Sheriff's Office on Cale Martin by leaving a copy and mailing a copy on 07/27/2018. jkg,notice of filing of petition for appointment of co-guardians and hearing thereon served b

In [28]:
ddf[ddf.c2dp_case_id == '82D04-2009-GU-000420']

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description,docket_event_comment,clean_comment
38,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579111846,38874260,C,C,QCSNEW,09/01/2020,Case Opened as a New Filing,NaN,NaN
39,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579112655,38874260,C,C,HSACT,09/01/2020,Hearing Scheduling Activity,Hearing on Petition to Establish Guardianship scheduled for 10/01/2020 at 8:20 AM.,Hearing on Petition to Establish Guardianship scheduled for at : AM
40,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579112865,38874260,C,ORGFD,PEG,09/01/2020,Petition to Establish Guardianship Filed,GR - Filed electronically,GR - Filed electronically
41,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579112899,38874260,C,C,APP,09/01/2020,Appearance Filed,NaN,NaN
42,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579113192,38874260,C,C,APP,09/01/2020,Appearance Filed,NaN,NaN
43,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579113712,38874260,C,MOT,MFEEW,09/01/2020,Motion for Fee Waiver Filed,NaN,NaN
44,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579114156,38874260,C,ORD,OGMFEEW,09/01/2020,Order Granting Motion for Fee Waiver,NaN,NaN
45,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579114628,38874260,C,OTHER,DOC,09/01/2020,Document Filed,hold hrg 10/1 proposed letters,hold hrg proposed letters
46,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579115600,38874260,C,OTHER,DOC,09/01/2020,Document Filed,hold hrg 10/1 letters of guardianship,hold hrg letters of guardianship
47,82D04-2009-GU-000420,18163,indiana___82D04-2009-GU-000420,indiana,vanderburgh,82d04-2009-gu-000420,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2020-09-01,NaN,NaN,NaN,pending,open/pending,False,579115601,38874260,C,OTHER,GUIS,09/01/2020,Guardianship Information Sheet Filed,NaN,NaN


In [11]:
ddf.docket_event_comment

149583

### Keyword Searches

In [8]:
ddf.head()

,c2dp_case_id,fips,case_key,state,county,case_number,raw_case_type,clean_case_type,date_filed,date_closed,date_disposed,disposition,raw_case_status,clean_case_status,is_closed,docket_eventkey,docket_casekey,docket_baseeventtype,docket_baseeventtypekey,docket_eventtype,docket_event_date,docket_event_description
0,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380563984,28090563,C,C,QCSNEW,08/02/2017,Case Opened as a New Filing
1,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564210,28090563,C,ORGFD,PEG,08/02/2017,Petition to Establish Guardianship Filed
2,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564214,28090563,C,ORGFD,PETG,08/02/2017,Petition for Emergency/Temporary Guardianship ...
3,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564217,28090563,C,OTHER,NF,08/02/2017,Notice Filed
4,02D03-1708-GU-000149,18003,indiana___02D03-1708-GU-000149,indiana,allen,02d03-1708-gu-000149,gu - guardianship - guardian: adult,gu - guardianship - guardian: adult,2017-08-02,2017-08-04,NaN,NaN,decided,disposed/closed,True,380564220,28090563,C,C,ADMOAF,08/02/2017,Oath Filed


Looking for "granted", "approved"

In [9]:
def search_keyword(wrds: list[str], persist=False): 
    kwdf = frequency_table
    for wd in wrds:
        kwdf = kwdf[kwdf.docket_event_description.str.lower().str.contains(wd, regex=True)]

    if persist:
        filename = "__and__".join([wrd.replace("(","").replace(")", "").replace("|", "_or_") for wrd in wrds])
        print(filename)
        kwdf.to_csv(f'../data/indiana/docket_keyword_search_results/{filename}.csv')
    return kwdf

In [10]:
search_keyword

<function __main__.search_keyword(wrds: list[str], persist=False)>

In [11]:
search_keyword(['(grant|approv|accept)'], persist=True)

grant_or_approv_or_accept


,docket_event_description,count,case_examples
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
25,Order Granting Motion to Terminate Guardianship,1546,49d08-1802-gu-004469; 49d08-2010-gu-034923; 49...
32,Order Granting Motion to Withdraw Appearance,1241,49d08-1711-gu-043504; 49d08-1705-gu-020694; 41...
36,Order Granting,1196,27d03-1608-gu-000058; 45d05-1905-gu-000147; 17...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
43,Order Granting Motion for Continuance,866,41d01-1708-gu-000131; 45d02-1905-gu-000146; 76...
45,Order Approving,819,49d08-1611-gu-040144; 42c01-1603-gu-000024; 49...
53,Order Granting Motion for Allowance of Fees,583,49d08-1810-gu-041476; 47c01-1609-gu-000065; 03...
57,Order Granting Motion for Fee Waiver,548,27d03-1804-gu-000030; 73d01-1612-gu-000070; 49...
61,Order Approving Biennial Accounting,468,52d02-1901-gu-000001; 49d08-1611-gu-041562; 53...


In [12]:
search_keyword(['(deny|reject)'], persist=True)

deny_or_reject


,docket_event_description,count,case_examples
117,Order Denying,95,49d08-1608-gu-029944; 49d08-1908-gu-033436; 49...
148,Order Denying Petition to Establish Guardianship,47,73c01-1801-gu-000002; 41d01-1909-gu-000171; 22...
175,Order Denying Motion for Continuance,32,49d08-1703-gu-011792; 79c01-2010-gu-000083; 45...
270,Order Denying Motion to Correct Error,6,49d08-2009-gu-033922; 49d08-1706-gu-023009; 49...
297,Order Denying Motion for Fee Waiver,5,52d01-1604-gu-000003; 28c01-1609-gu-000026; 79...
328,Order Denying Motion to Dismiss,4,49d08-1603-gu-008392; 49d08-1906-gu-022867; 49...
338,Order Denying Motion for Appointment,3,77c01-1908-gu-000027; 82d04-1602-gu-000083; 89...
345,Order Denying Motion to Consolidate,3,49d08-2007-gu-024495; 49d08-1711-gu-043747; 49...
354,Order Denying Motion to Terminate Guardianship,3,31c01-1611-gu-000055; 45d02-1807-gu-000138; 45...
382,Order Denying Motion to Reconsider,2,49d08-1605-gu-018488; 49d08-1906-gu-025473


In [13]:
search_keyword(['(guardian|conservator)'], persist=True)

guardian_or_conservator


,docket_event_description,count,case_examples
5,Petition to Establish Guardianship Filed,5152,29d03-1612-gu-000226; 10c04-1707-gu-000113; 02...
7,Letters of Guardianship Issued,5062,49d08-1811-gu-044988; 49d08-1612-gu-043009; 27...
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
9,Hearing on Petition to Establish Guardianship,4293,02d03-1810-gu-000278; 49d08-2011-gu-042052; 49...
17,Guardianship Information Sheet Filed,2102,02d03-2003-gu-000066; 17d02-2005-gu-000013; 49...
25,Order Granting Motion to Terminate Guardianship,1546,49d08-1802-gu-004469; 49d08-2010-gu-034923; 49...
33,Petition for Emergency/Temporary Guardianship ...,1238,45c01-1807-gu-000145; 49d08-2001-gu-001024; 49...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
42,Instruction Executed - Executor/Personal Rep/ ...,866,82d04-1612-gu-000682; 49d08-1607-gu-026735; 10...
46,Motion to Terminate Guardianship Filed,694,82d04-1610-gu-000571; 82d04-1704-gu-000216; 41...


In [14]:
search_keyword(['(guardian|conservator|gal|casa)','(grant|accept|deny|reject|approv|order)'], persist=True)

guardian_or_conservator_or_gal_or_casa__and__grant_or_accept_or_deny_or_reject_or_approv_or_order


,docket_event_description,count,case_examples
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
25,Order Granting Motion to Terminate Guardianship,1546,49d08-1802-gu-004469; 49d08-2010-gu-034923; 49...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
123,Order Granting Extension of Guardianship,84,49d08-1904-gu-013870; 49d08-1907-gu-029342; 49...
148,Order Denying Petition to Establish Guardianship,47,73c01-1801-gu-000002; 41d01-1909-gu-000171; 22...
238,Order Granting Motion to Remove Guardian,10,10c01-1612-gu-000191; 49d08-1804-gu-013870; 41...
354,Order Denying Motion to Terminate Guardianship,3,31c01-1611-gu-000055; 45d02-1807-gu-000138; 45...
388,Order Denying Motion to Remove Guardian,2,27d03-1706-gu-000051; 49d08-2008-gu-028430
435,Order Denying Extension of Guardianship,1,43d01-1809-gu-000071


In [15]:
search_keyword(['petition'], persist=True)

petition


,docket_event_description,count,case_examples
5,Petition to Establish Guardianship Filed,5152,29d03-1612-gu-000226; 10c04-1707-gu-000113; 02...
8,Order Granting Petition to Establish Guardianship,4443,53c01-1811-gu-000148; 82d04-1610-gu-000543; 18...
9,Hearing on Petition to Establish Guardianship,4293,02d03-1810-gu-000278; 49d08-2011-gu-042052; 49...
23,Petition Filed,1746,02d03-1812-gu-000331; 02d03-1804-gu-000086; 49...
33,Petition for Emergency/Temporary Guardianship ...,1238,45c01-1807-gu-000145; 49d08-2001-gu-001024; 49...
48,Hearing on Petition for Emergency Temporary Gu...,656,49d08-2009-gu-033413; 37c01-1608-gu-000646; 02...
59,Petition to Appoint GAL/CASA Filed,522,02d03-1604-gu-000096; 02d03-1704-gu-000060; 02...
148,Order Denying Petition to Establish Guardianship,47,73c01-1801-gu-000002; 41d01-1909-gu-000171; 22...
159,Petition for Appointment of Personal Represent...,37,41d01-1909-gu-000175; 02d03-1804-gu-000106; 41...
170,Petition for Extension of Temporary Guardianship,32,45c01-1809-gu-000215; 49d08-2001-gu-000430; 49...


In [16]:
search_keyword(['temporary'], persist=True)

temporary


,docket_event_description,count,case_examples
33,Petition for Emergency/Temporary Guardianship ...,1238,45c01-1807-gu-000145; 49d08-2001-gu-001024; 49...
37,Order Granting Temporary Guardianship,1101,48c01-1903-gu-000146; 49d08-1810-gu-039870; 10...
48,Hearing on Petition for Emergency Temporary Gu...,656,49d08-2009-gu-033413; 37c01-1608-gu-000646; 02...
170,Petition for Extension of Temporary Guardianship,32,45c01-1809-gu-000215; 49d08-2001-gu-000430; 49...
222,Temporary Appearance Filed,13,10c01-1712-gu-000187; 45c01-1802-gu-000026; 49...
299,Motion for Temporary Restraining Order Filed,5,79c01-2010-gu-000083; 49d08-2006-gu-021479; 37...
494,Order Granting Motion for Temporary Restrainin...,1,49d08-2006-gu-021479
502,Hearing on Motion for Temporary Restraining Order,1,49d08-2006-gu-021479


In [17]:
search_keyword(['(death|decease|dead|demise|died|funeral)'], persist=True)

death_or_decease_or_dead_or_demise_or_died_or_funeral


,docket_event_description,count,case_examples


In [18]:
frequency_table[frequency_table.docket_event_description.str.contains('Notice Filed')]

,docket_event_description,count,case_examples
14,Notice Filed,2283,02d03-2009-gu-000199; 49d08-1603-gu-009994; 46...
73,Waiver of Notice Filed,323,45d05-1808-gu-000177; 41d01-2008-gu-000131; 59...
311,Proof of Publication of Notice Filed,4,18c01-1811-gu-000140; 20d01-1609-gu-000123; 46...
350,Lis Pendens Notice Filed,3,80c01-1908-gu-000022; 49d08-1812-gu-048993


In [19]:
search_keyword(['dismiss'], persist=True)

dismiss


,docket_event_description,count,case_examples
68,Motion to Dismiss Filed,402,49d08-1707-gu-029295; 49d06-1904-gu-017473; 49...
74,Order of Dismissal,315,49d08-1606-gu-021955; 49d08-1910-gu-044622; 49...
89,Order Granting Motion to Dismiss,216,49d08-1606-gu-020109; 49d08-1701-gu-004265; 49...
142,Hearing on Motion for 41(E) Dismissal,52,34d04-1812-gu-000098; 49d08-2009-gu-031626; 49...
158,Hearing on Motion for 41E Dismissal,38,18c01-1606-gu-000060; 03c01-1709-gu-005029; 02...
221,Order of 41E Dismissal,13,71d05-1608-gu-000164; 55d01-1610-gu-000100; 27...
250,Dismissed,8,48c03-1906-gu-000348; 48c06-1710-gu-000486; 18...
252,Notice of Dismissal Filed,8,49d08-1610-gu-036977; 49d08-2009-gu-030536; 49...
266,Hearing on Motion to Dismiss,6,73d02-1604-gu-000023; 49d08-1712-gu-046127; 71...
269,Stipulation of Dismissal Filed,6,49d04-1605-gu-015747; 49d08-1712-gu-044650; 41...


### Missing Outcome Frequency Table